## CrypTen - Training an Encrypted Neural Network across Workers using Jails

In this tutorial we will train an encrypted neural network across different PySyft workers using Jails, an experimental mechanism into PySyft, we will be using CrypTen as a backend for SMPC. We use The MNIST dataset for this tutorial, the features will be split across alice and bob workers, each one will contain a set of pixels for every entry.

Authors:
- Ayoub Benaissa - Twitter: [@y0uben11](https://twitter.com/y0uben11)

### Setup

You should first know that you need to install PySyft from the `crypten` branch to be able to run the tutorial.

To prepare the dataset, you should run `./mnist_utils.py --option features --reduced 100 --binary` using the [mnist_utils.py from CrypTen](https://github.com/facebookresearch/CrypTen/blob/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py).

You should also start two GridNode with IDs alice and bob listening to ports 3000 and 3001 respectively, you should update the code if you change those information. For me I cloned the [PyGrid repo](https://github.com/OpenMined/PyGrid) and started the two GridNodes by running `./run.sh --id ID --port PORT --start_local_db` in separate terminals (you need to go in *apps/node*).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import crypten
import syft
from time import time
from syft.frameworks.crypten.context import run_multiworkers
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient


torch.manual_seed(0)
torch.set_num_threads(1)
hook = syft.TorchHook(torch)

We now define the neural network that we wanna use for training.

In [2]:
# Define an example network
class ExampleNet(nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 16 * 12 * 12)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

### Preparing remote workers

We now connect to alice and bob via their respective ports (update the url if you are running workers in a remote machine or used a different port), then prepare and send the data to the different workers. In a real scenario, the data would be already there stored privately.

In [3]:
# Syft workers
print("[%] Connecting to workers ...")
ALICE = DataCentricFLClient(hook, "ws://localhost:3000")
BOB = DataCentricFLClient(hook, "ws://localhost:3001")
print("[+] Connected to workers")

print("[%] Sending labels and training data ...")
# Prepare and send labels
label_eye = torch.eye(2)
labels = torch.load("/tmp/train_labels.pth")
labels = labels.long()
labels_one_hot = label_eye[labels]
labels_one_hot.tag("labels")
al_ptr = labels_one_hot.send(ALICE)
bl_ptr = labels_one_hot.send(BOB)

# Prepare and send training data
alice_train = torch.load("/tmp/alice_train.pth").tag("alice_train")
at_ptr = alice_train.send(ALICE)
bob_train = torch.load("/tmp/bob_train.pth").tag("bob_train")
bt_ptr = bob_train.send(BOB)

print("[+] Data ready")

[%] Connecting to workers ...
[+] Connected to workers
[%] Sending labels and training data ...
[+] Data ready


We now instanciate a model and create a dummy input that could be forwarded through it, this is needed to build the CrypTen model.

In [4]:
# Initialize model
dummy_input = torch.empty(1, 1, 28, 28)
pytorch_model = ExampleNet()

### Define and run the CrypTen computation using PySyft

Here we define the CrypTen computation for training the neural network, you only need to decorate it with `@run_multiworkers` to run the training across different workers.

In [5]:
@run_multiworkers([ALICE, BOB], master_addr="127.0.0.1", model=pytorch_model, dummy_input=dummy_input)
def run_encrypted_training():
    rank = crypten.communicator.get().get_rank()
    
    # Load the labels
    worker = syft.frameworks.crypten.get_worker_from_rank(rank)
    labels_one_hot = worker.search("labels")[0]

    # Load data:
    x_alice_enc = crypten.load("alice_train", 0)
    x_bob_enc = crypten.load("bob_train", 1)

    # Combine the feature sets: identical to Tutorial 3
    x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=2)

    # Reshape to match the network architecture
    x_combined_enc = x_combined_enc.unsqueeze(1)

    # model is sent from the master worker
    model.encrypt()
    # Set train mode
    model.train()

    # Define a loss function
    loss = crypten.nn.MSELoss()

    # Define training parameters
    learning_rate = 0.001
    num_epochs = 2
    batch_size = 10
    num_batches = x_combined_enc.size(0) // batch_size

    for i in range(num_epochs):
        # Print once for readability
        if rank == 0:
            print(f"Epoch {i} in progress:")
            pass

        for batch in range(num_batches):
            # define the start and end of the training mini-batch
            start, end = batch * batch_size, (batch + 1) * batch_size

            # construct AutogradCrypTensors out of training examples / labels
            x_train = x_combined_enc[start:end]
            y_batch = labels_one_hot[start:end]
            y_train = crypten.cryptensor(y_batch, requires_grad=True)

            # perform forward pass:
            output = model(x_train)

            loss_value = loss(output, y_train)

            # set gradients to "zero"
            model.zero_grad()

            # perform backward pass:
            loss_value.backward()

            # update parameters
            model.update_parameters(learning_rate)

            # Print progress every batch:
            batch_loss = loss_value.get_plain_text()
            if rank == 0:
                print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")

    model.decrypt()
    # printed contain all the printed strings during training
    return printed, model

And now we can start the distributed computation, `result` is a dictionary containing the result from every worker indexed by the rank of the party that they were running, so `result[0]` contains the result of the party 0 that was running in alice, `result[0][i]` contains the i'th returned value depending on how many values were returned.

In [6]:
print("[%] Starting computation")
func_ts = time()
result = run_encrypted_training()
func_te = time()
print(f"[+] run_encrypted_training() took {int(func_te - func_ts)}s")
printed = result[0][0]
model = result[0][1]
print(printed)

[%] Starting computation
[+] run_encrypted_training() took 45s
Epoch 0 in progress:
	Batch 1 of 10 Loss 0.4638
	Batch 2 of 10 Loss 0.4665
	Batch 3 of 10 Loss 0.4063
	Batch 4 of 10 Loss 0.3486
	Batch 5 of 10 Loss 0.3314
	Batch 6 of 10 Loss 0.2796
	Batch 7 of 10 Loss 0.2768
	Batch 8 of 10 Loss 0.2433
	Batch 9 of 10 Loss 0.2458
	Batch 10 of 10 Loss 0.2002
Epoch 1 in progress:
	Batch 1 of 10 Loss 0.1624
	Batch 2 of 10 Loss 0.1517
	Batch 3 of 10 Loss 0.1550
	Batch 4 of 10 Loss 0.1923
	Batch 5 of 10 Loss 0.1321
	Batch 6 of 10 Loss 0.1635
	Batch 7 of 10 Loss 0.2244
	Batch 8 of 10 Loss 0.1454
	Batch 9 of 10 Loss 0.1718
	Batch 10 of 10 Loss 0.1335



The model returned is a CrypTen model, but we can always run the usual PySyft methods to share the parameters and so on, as far as the model in not encrypted.

In [7]:
cp = syft.VirtualWorker(hook=hook, id="cp")
model.fix_prec()
model.share(ALICE, BOB, crypto_provider=cp)
print(model)
print(list(model.parameters())[0])

Graph unencrypted module
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:9377222447 -> alice:29717039925]
	-> [PointerTensor | me:41121311542 -> bob:57548505874]
	*crypto provider: cp*


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

he best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue"

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate
If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)

